# Generar un Campeonato

El propoósito del siguiente notebook es poder generar un campeonato aleatorio de $n$ equipos, donde cada equipo tenga un patrón de localías y visitas válido.

## Parametros

In [ ]:
team_amount = 8

assert team_amount % 2 == 0, "Equipos deben ser par"
assert team_amount <= 28, "Máxima cantidad de equipos 28"

## Equipos del campeonato

Se generan los equipos del campeonato. Los equipos serán las letras del abecedario.

In [ ]:
import string

alphabet = list(string.ascii_uppercase)

In [ ]:
teams = alphabet[0:team_amount]

In [ ]:
teams

['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H']

## Partidos

Se genera una lista de las combinaciónes de partidos a jugar.

In [ ]:
matches = []
for team_a in teams:
    for team_b in teams:
        if team_a != team_b:
            if team_a > team_b:
                matches.append(f"{team_a}-{team_b}")
            else:
                matches.append(f"{team_b}-{team_a}")

In [ ]:
matches = list(set(matches))

In [ ]:
matches = [x.split('-') for x in matches]

In [ ]:
matches[0:10]

[['F', 'E'],
 ['E', 'D'],
 ['D', 'B'],
 ['G', 'A'],
 ['C', 'A'],
 ['G', 'F'],
 ['B', 'A'],
 ['H', 'E'],
 ['G', 'C'],
 ['H', 'C']]

In [ ]:
matches_per_date = len(teams) / 2

In [ ]:
dates_number = int(len(matches) / matches_per_date)

## Patrones

Se genera un conjuntos de patrones que sea valido para $n$ equipos. Un conjunto de patrones es valido si para cada fecha la cantidad de visitas es igual a la de localias

In [ ]:
import itertools

In [ ]:
patterns = ["".join(seq) for seq in itertools.product("01", repeat=dates_number)]
patterns = filter(lambda x: x.count('000') == 0, patterns)
patterns = filter(lambda x: x.count('111') == 0, patterns)
patterns = filter(lambda x: x.count('00') <= 1, patterns)
patterns = filter(lambda x: x.count('11') <= 1, patterns)

patterns = list(patterns)
patterns[:5]

['0010101', '0010110', '0011010', '0100101', '0100110']

In [ ]:
def check_pattern_group(patterns):
    """
    Retorna true si el set de patrones es válido, es decir,
    para cada fecha la cantidad de localias y visitas es igual.
    """
    for i in range(len(patterns[0])):
        local = 0
        visit = 0
        for pattern in patterns:
            if pattern[i] == '1':
                local += 1
            else: visit += 1
        if local != visit: return False
    return True

Se obtiene un sample aleatorio de patrones, luego se itera hasta que el sample sea válido.

In [ ]:
import random
import time

start = time.time()
iter = 0

while True:
    iter += 1
    patterns_sample = random.sample(patterns, len(teams))
    if check_pattern_group(patterns_sample):
        break

print('Tiempo:', time.time() - start)
print('Iteraciones:', iter)

Tiempo: 0.019544601440429688
Iteraciones: 891


In [ ]:
patterns_sample

['1010101',
 '0101101',
 '1101001',
 '1101010',
 '0110010',
 '1001010',
 '0010101',
 '0010110']

Se le asigna un patrón a cada equipo

In [ ]:
team_patterns = {team: pat for team, pat in zip(teams, patterns_sample)}

In [ ]:
team_patterns

{'A': '1010101',
 'B': '0101101',
 'C': '1101001',
 'D': '1101010',
 'E': '0110010',
 'F': '1001010',
 'G': '0010101',
 'H': '0010110'}

## Primera vuelta

Ahora, se asignan las fechas

In [ ]:
def team_in_date(team, date_matches):
    """
    Retorna true si el equipo esta dentro de
    los partidos de la fecha
    """
    teams_in_date = []
    for a, b in date_matches:
        teams_in_date.append(a)
        teams_in_date.append(b)
    return team in teams_in_date

In [ ]:
championship = {}

for date in range(1, dates_number + 1):
    date_matches = []
    for team in teams:
        # Si el equipo ya fue agregado, se pasa al siguiente
        if team_in_date(team, date_matches):
            continue
        if len(date_matches) == len(teams) / 2:
            break
        # se revisa si al equipo le toca de local o visita
        local = team_patterns[team][date - 1] == '1'
        # se selecciona su rival de manera aleatoria.
        possible_teams = []
        for team2 in teams:
            # No puede ser el equipo actual
            if team == team2:
                continue
            # No puede estar en ninguna fecha
            if team_in_date(team2, date_matches):
                continue
            team2_local = team_patterns[team2][date - 1] == '1'
            # No debe ser local si el equipo actual es local o viceversa
            if local == (not team2_local):
                possible_teams.append(team2)
        team2 = random.choice(possible_teams)
        if local:
            date_matches.append((team, team2))
        else:
            date_matches.append((team2, team))

    championship[date] = date_matches

In [ ]:
championship

{1: [('A', 'G'), ('C', 'B'), ('D', 'H'), ('F', 'E')],
 2: [('B', 'A'), ('C', 'G'), ('D', 'H'), ('E', 'F')],
 3: [('A', 'F'), ('G', 'B'), ('H', 'C'), ('E', 'D')],
 4: [('F', 'A'), ('B', 'H'), ('C', 'G'), ('D', 'E')],
 5: [('A', 'F'), ('B', 'E'), ('G', 'C'), ('H', 'D')],
 6: [('E', 'A'), ('D', 'B'), ('H', 'C'), ('F', 'G')],
 7: [('A', 'F'), ('B', 'E'), ('C', 'D'), ('G', 'H')]}

## Segunda vuelta

Ahora, asignarle patrones a los equipos es un poco más complicado, dado que un equipo tiene localías faltantes, y puede arrastrar un break.

In [ ]:
team_patterns_2 = {}
memo = {}

def get_team_pattern(team):
    if team in memo.keys():
        patterns_c = memo[team]
    else:
        pat = team_patterns[team]
        visit = pat.count('0')
        last_2_dates = pat[-2:]
        patterns_c = patterns.copy()
        patterns_c = [last_2_dates + p for p in patterns_c]
        # se filtran si tiene 000 o 111
        patterns_c = filter(lambda x: x.count('000') == 0, patterns_c)
        patterns_c = filter(lambda x: x.count('111') == 0, patterns_c)

        patterns_c = [x[1:] for x in patterns_c]
        # se filtran si tienen mas de dos breaks
        patterns_c = filter(lambda x: x.count('00') <= 1, patterns_c)
        patterns_c = filter(lambda x: x.count('11') <= 1, patterns_c)
        patterns_c = [x[1:] for x in patterns_c]

        # visitas restantes iguales
        patterns_c = filter(lambda x: x.count('0') <= visit, patterns_c)

        patterns_c = list(patterns_c)
        memo[team] = patterns_c
    return random.choice(patterns_c)

In [ ]:
start = time.time()
iter = 0

while True:
    iter += 1
    patterns_sample = []
    for team in teams:
        patterns_sample.append(get_team_pattern(team))
    if check_pattern_group(patterns_sample):
        break

print('Tiempo:', time.time() - start)
print('Iteraciones:', iter)

Tiempo: 0.05444002151489258
Iteraciones: 1420


In [ ]:
team_patterns = {team: pat for team, pat in zip(teams, patterns_sample)}

Se agrega la segunda vuelta al campeonato

In [ ]:
for date in range(1, dates_number + 1):
    date_matches = []
    for team in teams:
        # Si el equipo ya fue agregado, se pasa al siguiente
        if team_in_date(team, date_matches):
            continue
        if len(date_matches) == len(teams) / 2:
            break
        # se revisa si al equipo le toca de local o visita
        local = team_patterns[team][date - 1] == '1'
        # se selecciona su rival de manera aleatoria.
        # para esto, el equipo no debe haber jugado en la fecha, y
        # debe ser visita si el primer equipo es local o viceversa
        possible_teams = []
        for team2 in teams:
            if team == team2:
                continue
            if team_in_date(team2, date_matches):
                continue
            team2_local = team_patterns[team2][date - 1] == '1'
            if local == (not team2_local):
                possible_teams.append(team2)
        team2 = random.choice(possible_teams)
        if local:
            date_matches.append((team, team2))
        else:
            date_matches.append((team2, team))

    championship[date + 5] = date_matches

In [ ]:
championship

{1: [('A', 'G'), ('C', 'B'), ('D', 'H'), ('F', 'E')],
 2: [('B', 'A'), ('C', 'G'), ('D', 'H'), ('E', 'F')],
 3: [('A', 'F'), ('G', 'B'), ('H', 'C'), ('E', 'D')],
 4: [('F', 'A'), ('B', 'H'), ('C', 'G'), ('D', 'E')],
 5: [('A', 'F'), ('B', 'E'), ('G', 'C'), ('H', 'D')],
 6: [('B', 'A'), ('H', 'C'), ('G', 'D'), ('E', 'F')],
 7: [('A', 'E'), ('F', 'B'), ('C', 'G'), ('D', 'H')],
 8: [('H', 'A'), ('B', 'E'), ('G', 'C'), ('D', 'F')],
 9: [('A', 'D'), ('C', 'B'), ('E', 'G'), ('F', 'H')],
 10: [('D', 'A'), ('B', 'H'), ('C', 'E'), ('G', 'F')],
 11: [('A', 'B'), ('H', 'C'), ('F', 'D'), ('E', 'G')],
 12: [('A', 'E'), ('B', 'H'), ('C', 'F'), ('D', 'G')]}

## Puntos

Por último, falta asignarle puntaje a los partidos. La distribución de los goles de un partido es totalmente arbitraria.

In [ ]:
possible_points = [0] * 6 + [1] * 5 + [2] * 4 + [3] * 3 + [4] * 2 + [5]
possible_points

[0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 2, 2, 2, 2, 3, 3, 3, 4, 4, 5]

In [ ]:
points_range = list(set(possible_points))
points_range.sort()

for p in list(set(possible_points)):
    count = possible_points.count(p)
    print(f'P(X={p}) =', round(count / len(possible_points), 3))

P(X=0) = 0.286
P(X=1) = 0.238
P(X=2) = 0.19
P(X=3) = 0.143
P(X=4) = 0.095
P(X=5) = 0.048


In [ ]:
def get_score():
    return random.choice(possible_points)

In [ ]:
final_champ = {}

for date, matches_list in championship.items():
    matches = []
    for local, visit in matches_list:
        matches.append({
            'local': local,
            'visit': visit,
            'score': f"{get_score()}:{get_score()}"
        })
    final_champ[date] = matches

In [ ]:
final_champ

{1: [{'local': 'A', 'score': '1:2', 'visit': 'G'},
  {'local': 'C', 'score': '4:1', 'visit': 'B'},
  {'local': 'D', 'score': '1:4', 'visit': 'H'},
  {'local': 'F', 'score': '5:3', 'visit': 'E'}],
 2: [{'local': 'B', 'score': '3:1', 'visit': 'A'},
  {'local': 'C', 'score': '0:0', 'visit': 'G'},
  {'local': 'D', 'score': '0:0', 'visit': 'H'},
  {'local': 'E', 'score': '1:4', 'visit': 'F'}],
 3: [{'local': 'A', 'score': '3:1', 'visit': 'F'},
  {'local': 'G', 'score': '0:2', 'visit': 'B'},
  {'local': 'H', 'score': '2:2', 'visit': 'C'},
  {'local': 'E', 'score': '1:0', 'visit': 'D'}],
 4: [{'local': 'F', 'score': '2:5', 'visit': 'A'},
  {'local': 'B', 'score': '0:1', 'visit': 'H'},
  {'local': 'C', 'score': '1:0', 'visit': 'G'},
  {'local': 'D', 'score': '0:0', 'visit': 'E'}],
 5: [{'local': 'A', 'score': '3:1', 'visit': 'F'},
  {'local': 'B', 'score': '1:3', 'visit': 'E'},
  {'local': 'G', 'score': '1:1', 'visit': 'C'},
  {'local': 'H', 'score': '1:1', 'visit': 'D'}],
 6: [{'local': 'B', 

In [ ]:
last_date = max(championship.keys())
last_first_round_date = int(last_date / 2)

In [ ]:
team_first_round_points = {}
localies_left = {}

for team in teams:
    points = 0
    localies = 0
    for i in range(last_first_round_date):
        i += 1
        filt = lambda x: team in x['local'] or team in x['visit']
        match = list(filter(filt, final_champ[i]))[0]
        if match['local'] == team and match['score'][0] > match['score'][2]:
            points += 3
        if match['visit'] == team and match['score'][0] < match['score'][2]:
            points += 3
        if match['score'][0] == match['score'][2]:
            points += 1
        if match['local'] == team:
            localies += 1
    team_first_round_points[team] = points
    localies_left[team] = last_first_round_date - localies

In [ ]:
localies_left

{'A': 3, 'B': 2, 'C': 3, 'D': 3, 'E': 3, 'F': 4, 'G': 3, 'H': 3}

In [ ]:
team_first_round_points

{'A': 9, 'B': 9, 'C': 10, 'D': 6, 'E': 7, 'F': 9, 'G': 5, 'H': 10}

## Se escribe el archivo xlsx

In [ ]:
!pip install XlsxWriter

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 149 kB 5.2 MB/s 


In [ ]:
import xlsxwriter

workbook = xlsxwriter.Workbook(f'campeonato_{team_amount}.xlsx')

## Hoja 1

In [ ]:
worksheet = workbook.add_worksheet('Equipos')

In [ ]:
for i in range(2, len(teams) + 2):
    worksheet.write(f'A{i}', i - 1)

In [ ]:
worksheet.write('B1', 'EQUIPO')
worksheet.write('C1', 'ALIAS')
worksheet.write('D1', 'Puntos al finalizar la primera rueda')
worksheet.write('E1', 'Localías faltantes')

0

In [ ]:
for i in range(2, len(teams) + 2):
    worksheet.write(f'B{i}', teams[i - 2])
    worksheet.write(f'C{i}', teams[i - 2])
    worksheet.write(f'D{i}', team_first_round_points[teams[i - 2]])
    worksheet.write(f'E{i}', localies_left[teams[i - 2]])

## Hoja 2

In [ ]:
worksheet = workbook.add_worksheet('Resultados')

In [ ]:
worksheet.write('A1', 'Jornada')
worksheet.write('B1', 'Fecha')
worksheet.write('C1', 'Local')
worksheet.write('D1', 'Visita')
worksheet.write('E1', 'Resultado')

0

In [ ]:
dates_rev = list(final_champ.keys())
dates_rev.sort(reverse=True)

row = 2
for date in dates_rev:
    worksheet.write(f'A{row}', date)
    row += 1
    for match in final_champ[date]:
        worksheet.write(f'B{row}', '1/1/2022')
        worksheet.write(f'C{row}', match['local'])
        worksheet.write(f'D{row}', match['visit'])
        worksheet.write(f'E{row}', match['score'])
        row += 1

In [ ]:
workbook.close()